# Import modules

In [1]:
import os
import pickle
import importlib
import subprocess
import pandas as pd

try:
    importlib.import_module('pygrpm')
except ImportError:
    subprocess.check_call(["pip", "install", "git+https://github.com/johndef64/GRPM_system.git"])
    
from pygrpm import *

# GET Datasets

In [4]:
### GET Datasets ###
if not os.path.exists('grpm_dataset/grpm_dataset.parquet'):
    get_and_extract('grpm_dataset', record_id='14052302')
    get_and_extract('nutrigenetic_dataset', record_id='14052302')

# LOAD Datasets

In [2]:
### LOAD Datasets ###
pcg_grpm, rna_grpm, pseudo_grpm = grpm_importer()
grpm_nutrigen, grpm_nutrigen_int, grpm_nutrigen_int_gwas = nutrig_importer()

display(grpm_nutrigen_int)

Importing time:  0:00:06.737058
pcg: 776.19 MB
rna: 58.18 MB
pseudo: 1.93 MB
nutrigen dataset: 87.13 MB
nutrigen dataset filtered: 53.62 MB
nutrigen gwas dataset: 20.56 MB


,gene,rsid,pmid,mesh,topic,interest_index
0,FTO,rs9972653,32393786,Body Mass Index,General Nutrition,1.00000
1,FTO,rs9972653,32393786,"Diabetes Mellitus, Type 2",General Nutrition,1.00000
2,FTO,rs9972653,32393786,Diet,General Nutrition,1.00000
3,FTO,rs9972653,33128006,Body Mass Index,General Nutrition,1.00000
4,FTO,rs9972653,33128006,Cardiovascular Diseases,General Nutrition,1.00000
...,...,...,...,...,...,...
1171244,FADS1,rs174545,23221573,Lipid Metabolism,Xenobiotics Metabolism,0.01255
1171245,FADS1,rs174545,30120404,Liver,Xenobiotics Metabolism,0.01255
1171246,FADS1,rs174545,30120404,Non-alcoholic Fatty Liver Disease,Xenobiotics Metabolism,0.01255
1171247,FADS1,rs174544,20565855,Lipid Metabolism,Xenobiotics Metabolism,0.01255


# SHOW Stats

In [ ]:
%%time
pcg_grpm_stats = get_stats(pcg_grpm, group_by = 'gene')
display(pcg_grpm_stats)

Computing Stats...


In [ ]:
%%time
grpm_nutrigen_stats = get_stats(grpm_nutrigen, group_by = 'gene')
display(grpm_nutrigen_stats)

# QUERY GRPM Dataset 

## MeSH Query Example

In [3]:
## GET MeSH Dataset ##
if not os.path.exists('ref-mesh/MESH_STY_LITVAR1.csv'):
    get_and_extract('ref-mesh', record_id='14052302')
    get_topic_terms()

In [4]:
# LOAD MeSH
grpm_mesh = mesh_importer()
grpm_mesh.head()

GRPM MeSH count: 21705
semantic types: 125


,Preferred Label,Semantic Types Label,Class ID,mesh_id,Semantic Types
1,Electronic Health Records,Intellectual Product,http://purl.bioontology.org/ontology/MESH/D057286,D057286,T170
2,Consent Forms,Intellectual Product,http://purl.bioontology.org/ontology/MESH/D032962,D032962,T170
3,Genealogy and Heraldry,Intellectual Product,http://purl.bioontology.org/ontology/MESH/D005789,D005789,T170
4,Publications,Intellectual Product,http://purl.bioontology.org/ontology/MESH/D011642,D011642,T170
5,Pharmaceutical Services,Intellectual Product,http://purl.bioontology.org/ontology/MESH/D010593,D010593,T170


In [5]:
get_stats(grpm_mesh, "Semantic Types Label")

Computing Stats...
runtime:  0:00:00.646001


,Preferred Label,Class ID,mesh_id,Semantic Types
,unique,unique,unique,unique
Semantic Types Label,,,,
Acquired Abnormality,41,41,41,1
Activity,38,38,38,1
Age Group,15,15,15,1
Amino Acid Sequence,45,45,45,1
"Amino Acid, Peptide, or Protein",4625,4625,4625,1
...,...,...,...,...
Therapeutic or Preventive Procedure,693,693,693,1
Tissue,89,89,89,1


In [8]:
# Random Query Example
mesh_query =  grpm_mesh['Preferred Label'].drop_duplicates().sample(10).to_list()

# Filter and get unique results
result = query_dataset(pcg_grpm, mesh_query, 'mesh')
display(result)

,gene,type,rsid,pmid,mesh,qualifier,major
38033,MT-CO3,PCG,rs2853826,34576000,Epigenome,None,True
45210,MT-ND3,PCG,rs2853826,34576000,Epigenome,None,True
52523,MT-ND4L,PCG,rs2853826,34576000,Epigenome,None,True
62152,MT-ND4,PCG,rs2853826,34576000,Epigenome,None,True
73836,MT-ND5,PCG,rs2853826,34576000,Epigenome,None,True
...,...,...,...,...,...,...,...
16595876,CXCL8,PCG,rs2227532,29105764,Interleukin-1,genetics,False
16595895,CXCL8,PCG,rs4073,29105764,Interleukin-1,genetics,False
16597205,CXCL8,PCG,rs2227306,16719905,Interleukin-1,genetics,False
16597221,CXCL8,PCG,rs4073,16719905,Interleukin-1,genetics,False


## Build MeSH Query [CUDA recommended]
In Colab:: load Runtime with GPU 

In [ ]:
import torch
print("Torch version:",torch.__version__)
print("Is CUDA enabled?",torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.randn(1).cuda())
    
# if not CUDA and Windows system: unintall torch and install "pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118"

In [108]:
import torch
# LOAD Language Model
MODEL = 'dmis-lab/biobert-v1.1'
model = load_language_model(MODEL)
file_path = 'ref-mesh/GrpmMeshEmbeddings_biobert-v1.1.pkl'

# Get MeSH embeddings
if torch.cuda.is_available() and not os.path.isfile(file_path):
    # Generate MeSH Embeddings
    grpm_meshes = list(grpm_mesh['Preferred Label'].drop_duplicates().reset_index(drop=True))
    mesh_embeddings = extract_embedding(grpm_meshes, model)
    grpm_mesh_embeddings = {"meshes":grpm_meshes, "embeddings":mesh_embeddings}

    # Open the file in write-binary mode to store the pickle
    with open(file_path, 'wb') as file:
        # Use pickle to dump the dictionary into the file
        pickle.dump(grpm_mesh_embeddings, file)
else:
    # Download and import MeSH Embeddings
    grpm_mesh_embeddings = import_mesh_embeddings()
    
grpm_meshes = grpm_mesh_embeddings['meshes']
mesh_embeddings = grpm_mesh_embeddings['embeddings']

No sentence-transformers model found with name dmis-lab/biobert-v1.1. Creating a new one with mean pooling.


In [91]:
# User defined Topic Terms

user_query = "diet ketogenic, diet reducing, diet sodium-restricted, diet, dietary, dietetics, dyslipidemias, eating disorders, feeding and eating disorde, food hypersensitivity, foodborne diseases, gastrointestinal diseases, hypercholesterolemia, hyperglycemia, hyperlipidemias, hyperphagia, hypoglycemia, hypophagia, insulin resistance"  # comma separated list

topic_terms_list = user_query.split(',')
topic_terms = pd.Series(topic_terms_list)

In [ ]:
# Extract MeSH Query
tab = create_corr_table(topic_terms, grpm_meshes, model, mesh_embeddings)

threshold = 0.90 # set similarity threshold
mesh_query = tab[tab.similarity >= threshold].list2.to_list()
print('\n\nMeSH Query:', mesh_query)

## Execute MeSH Query

In [112]:
# Filter and get unique results
result = query_dataset(pcg_grpm, mesh_query, 'mesh')
display(result)

,gene,type,rsid,pmid,mesh,qualifier,major
5434,MT-ND1,PCG,rs200319905,25579139,Hyperlipidemias,genetics,False
5435,MT-ND1,PCG,rs200319905,25579139,Hyperlipidemias,metabolism,False
5436,MT-ND1,PCG,rs200319905,25579139,Hyperlipidemias,pathology,False
13642,MT-ND2,PCG,rs200319905,25579139,Hyperlipidemias,genetics,False
13643,MT-ND2,PCG,rs200319905,25579139,Hyperlipidemias,metabolism,False
...,...,...,...,...,...,...,...
16606654,CXCL8,PCG,rs572157399,20508232,Hypoglycemia,complications,False
16606655,CXCL8,PCG,rs572157399,20508232,Hypoglycemia,diagnosis,False
16606656,CXCL8,PCG,rs572157399,20508232,Hypoglycemia,epidemiology,True
16607143,CXCL8,PCG,rs572157399,21357364,Hyperglycemia,metabolism,False


# QUERY Nutrigenetic Dataset

In [113]:
get_stats(grpm_nutrigen_int, 'topic')

Computing Stats...
runtime:  0:00:01.267998


,gene,rsid,pmid,mesh
,unique,unique,unique,unique
topic,,,,
Cardiovascular Health and Lipid Metabolism,975,41931,66113,521
Diabetes Mellitus Type II and Metabolic Syndrome,603,22270,36198,297
Diet-induced Oxidative Stress,75,2559,10058,60
Eating Behavior and Taste Sensation,211,4252,7241,256
Food Allergies,451,6289,7762,64
Food Intolerances,392,5008,4726,125
General Nutrition,686,26456,44859,397
"Obesity, Weight Control and Compulsive Eating",317,10842,22123,230


In [114]:
# Select Topic
topic = "Vitamin and Micronutrients Metabolism and Deficiency-Related Diseases"

# Filter and get unique results
topic_data = query_dataset(grpm_nutrigen_int, [topic], 'topic')
print(f'Displaying "{topic}" topic')
display(topic_data)

Displaying "Vitamin and Micronutrients Metabolism and Deficiency-Related Diseases" topic


,gene,rsid,pmid,mesh,topic,interest_index
967603,VDR,rs987849,34578986,Nutritional Status,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967604,VDR,rs987849,34578986,"Receptors, Calcitriol",Vitamin and Micronutrients Metabolism and Defi...,1.00000
967605,VDR,rs987849,34578986,Vitamin D,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967606,VDR,rs987849,34578986,Vitamin D Deficiency,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967607,VDR,rs987849,34578986,Vitamin D-Binding Protein,Vitamin and Micronutrients Metabolism and Defi...,1.00000
...,...,...,...,...,...,...
1002198,ITPR1,rs121912425,21555639,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002199,ITPR1,rs121912425,22986007,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002200,ITPR1,rs121912425,18579805,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002201,ITPR1,rs121912425,27108798,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255


In [115]:
# Get Topic Data Stats
stats = get_stats(topic_data, "gene")
# Create a sorted index by finding the order of elements in 'gene_sorted_gi'
gene_sorted_gi = track_data.gene.drop_duplicates().to_list()
sorted_index = stats.index.map(lambda x: gene_sorted_gi.index(x) if x in gene_sorted_gi else float('inf'))

# Sort 'stats' based on the created sorted index
stats_sorted = stats.iloc[sorted_index.argsort()]
stats_sorted

Computing Stats...
runtime:  0:00:00.404005


,rsid,pmid,mesh,topic
,unique,unique,unique,unique
gene,,,,
VDR,185,922,41,1
MTHFR,105,1622,79,1
GSTP1,25,711,29,1
GC,76,476,28,1
CYP2R1,25,289,20,1
...,...,...,...,...
SLC19A3,18,20,10,1
ACADM,51,43,14,1


## 1. Gene Query Example

In [117]:
# Gene Query on Nutrigenetic Topic
my_genes = "VDR, G6PD, GSTP1"

# Filter and get unique results
result = query_dataset(topic_data, my_genes.split(','), 'gene')
display(result)

,gene,rsid,pmid,mesh,topic,interest_index
967603,VDR,rs987849,34578986,Nutritional Status,Vitamin and Micronutrients Metabolism and Defi...,1.0
967604,VDR,rs987849,34578986,"Receptors, Calcitriol",Vitamin and Micronutrients Metabolism and Defi...,1.0
967605,VDR,rs987849,34578986,Vitamin D,Vitamin and Micronutrients Metabolism and Defi...,1.0
967606,VDR,rs987849,34578986,Vitamin D Deficiency,Vitamin and Micronutrients Metabolism and Defi...,1.0
967607,VDR,rs987849,34578986,Vitamin D-Binding Protein,Vitamin and Micronutrients Metabolism and Defi...,1.0
...,...,...,...,...,...,...
973278,VDR,rs10083198,34578986,Vitamin D Deficiency,Vitamin and Micronutrients Metabolism and Defi...,1.0
973279,VDR,rs10083198,34578986,Vitamin D-Binding Protein,Vitamin and Micronutrients Metabolism and Defi...,1.0
973280,VDR,rs10083198,29380949,"Receptors, Calcitriol",Vitamin and Micronutrients Metabolism and Defi...,1.0
973281,VDR,rs10083198,29380949,Vitamin D,Vitamin and Micronutrients Metabolism and Defi...,1.0


In [ ]:
# Gene Query on Nutrigenetic-GWAS ds
result = query_dataset(grpm_nutrigen_int_gwas, my_genes, 'GRPM_GENE')
display(result)

## 3. Advanced query example

Let's explore the Relationship between nutritional status (micronutrients) and the systemic inflammatory response at genetic level:  \cite{https://pubmed.ncbi.nlm.nih.gov/30220267/}

Build a composite query:
1. Impact of Systemic Inflammatory Response on Plasma Micronutrient Levels
2. Methods for Adjusting Micronutrient Measurements in the Presence of Inflammation

In [130]:
QUERIES

['',
 'Impact of Systemic Inflammatory Response on Plasma Micronutrient Levels',
 'Methods for Adjusting Micronutrient Measurements in the Presence of Inflammation',
 '']

In [133]:
from pygrpm import *

# Download and import MeSH Embeddings
grpm_mesh_embeddings = import_mesh_embeddings()

grpm_meshes = grpm_mesh_embeddings['meshes']
mesh_embeddings = grpm_mesh_embeddings['embeddings']

QUERY = "systemic inflammatory response"
QUERIES = """
Impact of Systemic Inflammatory Response on Plasma Micronutrient Levels;
Methods for Adjusting Micronutrient Measurements in the Presence of Inflammation
""".split(";")

# QUERY = "Cardiovascualr disease"
composite_query = []
for query in QUERIES:
    mesh_rankings = get_mesh_rankings(query,
                      grpm_meshes,
                      model, mesh_embeddings=mesh_embeddings)
    threshold = 0.85
    mesh_query = filter_mesh_scores(mesh_rankings, threshold)
    composite_query.extend(mesh_query)

composite_query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Relevant terms: ['Calcium-Regulating Hormones and Agents', 'Anti-Inflammatory Agents', 'Anti-Inflammatory Agents, Non-Steroidal', 'Melanocyte-Stimulating Hormones', 'Inducible T-Cell Co-Stimulator Protein', 'Oxidative Phosphorylation Coupling Factors', 'Intracellular Calcium-Sensing Proteins', 'Immune Checkpoint Proteins', 'Plasma Membrane Calcium-Transporting ATPases', 'Immunoglobulins, Thyroid-Stimulating', 'Glycine Plasma Membrane Transport Proteins', 'Serotonin Plasma Membrane Transport Proteins', 'Norepinephrine Plasma Membrane Transport Proteins', 'Plasma Membrane Neurotransmitter Transport Proteins', 'Catecholamine Plasma Membrane Transport Proteins', 'GABA Plasma Membrane Transport Proteins', 'Dopamine Plasma Membrane Transport Proteins', 'Pulmonary Surfactant-Associated Protein A', 'Pulmonary Surfactant-Associated Proteins', 'Immunoglobulin J Recombination Signal Sequence-Binding Protein', 'Seminal Plasma Proteins', 'Pregnancy-Associated Plasma Protein-A', 'Equilibrative Nucle

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Relevant terms: ['Response Evaluation Criteria in Solid Tumors', 'Anti-Inflammatory Agents, Non-Steroidal', 'Intracellular Calcium-Sensing Proteins', 'Norepinephrine Plasma Membrane Transport Proteins', 'Plasma Membrane Neurotransmitter Transport Proteins', 'Equilibrative Nucleoside Transport Proteins', 'Soluble N-Ethylmaleimide-Sensitive Factor Attachment Proteins', 'Salivary Proteins and Peptides', 'Neuronal Calcium-Sensor Proteins', 'Circadian Rhythm Signaling Peptides and Proteins', 'High-Temperature Requirement A Serine Peptidase 2', 'High-Temperature Requirement A Serine Peptidase 1', 'Dietary Approaches To Stop Hypertension', 'Calcium Metabolism Disorders', 'Congenital Disorders of Glycosylation', 'Autoimmune Diseases of the Nervous System', 'Steroid Metabolism, Inborn Errors', 'Bisphosphonate-Associated Osteonecrosis of the Jaw', 'Simian Acquired Immunodeficiency Syndrome', 'Metal Metabolism, Inborn Errors', 'Central Nervous System Bacterial Infections', 'Feline Acquired Immuno

['Calcium-Regulating Hormones and Agents',
 'Anti-Inflammatory Agents',
 'Anti-Inflammatory Agents, Non-Steroidal',
 'Melanocyte-Stimulating Hormones',
 'Inducible T-Cell Co-Stimulator Protein',
 'Oxidative Phosphorylation Coupling Factors',
 'Intracellular Calcium-Sensing Proteins',
 'Immune Checkpoint Proteins',
 'Plasma Membrane Calcium-Transporting ATPases',
 'Immunoglobulins, Thyroid-Stimulating',
 'Glycine Plasma Membrane Transport Proteins',
 'Serotonin Plasma Membrane Transport Proteins',
 'Norepinephrine Plasma Membrane Transport Proteins',
 'Plasma Membrane Neurotransmitter Transport Proteins',
 'Catecholamine Plasma Membrane Transport Proteins',
 'GABA Plasma Membrane Transport Proteins',
 'Dopamine Plasma Membrane Transport Proteins',
 'Pulmonary Surfactant-Associated Protein A',
 'Pulmonary Surfactant-Associated Proteins',
 'Immunoglobulin J Recombination Signal Sequence-Binding Protein',
 'Seminal Plasma Proteins',
 'Pregnancy-Associated Plasma Protein-A',
 'Equilibrative

In [134]:
composite_query

['Calcium-Regulating Hormones and Agents',
 'Anti-Inflammatory Agents',
 'Anti-Inflammatory Agents, Non-Steroidal',
 'Melanocyte-Stimulating Hormones',
 'Inducible T-Cell Co-Stimulator Protein',
 'Oxidative Phosphorylation Coupling Factors',
 'Intracellular Calcium-Sensing Proteins',
 'Immune Checkpoint Proteins',
 'Plasma Membrane Calcium-Transporting ATPases',
 'Immunoglobulins, Thyroid-Stimulating',
 'Glycine Plasma Membrane Transport Proteins',
 'Serotonin Plasma Membrane Transport Proteins',
 'Norepinephrine Plasma Membrane Transport Proteins',
 'Plasma Membrane Neurotransmitter Transport Proteins',
 'Catecholamine Plasma Membrane Transport Proteins',
 'GABA Plasma Membrane Transport Proteins',
 'Dopamine Plasma Membrane Transport Proteins',
 'Pulmonary Surfactant-Associated Protein A',
 'Pulmonary Surfactant-Associated Proteins',
 'Immunoglobulin J Recombination Signal Sequence-Binding Protein',
 'Seminal Plasma Proteins',
 'Pregnancy-Associated Plasma Protein-A',
 'Equilibrative

Gat Genes and Variants possibly realted to the Query

In [ ]:
# Filter and get unique results
result = query_dataset(grpm_nutrigen_int, mesh_query, 'mesh')
display(result)

In [127]:
mendel.c("""@
Your task is to extract two main topics in the form of a compact and meaningful query from this work below:

"""+TITABS, 4)

*chat cleared*
1. Impact of Systemic Inflammatory Response on Plasma Micronutrient Levels
2. Methods for Adjusting Micronutrient Measurements in the Presence of Inflammation

In [126]:
TITABS = """
Relationship between nutritional status and the systemic inflammatory response: micronutrients

Abstract
Micronutrients such as trace elements and vitamins are important as enzyme cofactors in the metabolism of all cells in the body and therefore key to determining nutritional status. The present systematic review examined the evidence of the impact of the systemic inflammatory response on plasma micronutrient status in acute (surgical) and chronic tissue injury. A literature review using targeted subject headings was carried out. Plasma C-reactive protein was used to classify minor (80 mg/l) inflammation. The literature search produced 2344 publications and plasma vitamin D, zinc and carotenoids were most commonly studied and plasma vitamins K, B2 and B6 were least studied. In acute injury thirteen studies (all prospective) and in chronic injury twenty-four studies (largely retrospective) were included in the review. There was consistent evidence that most common measured micronutrients in the plasma (zinc, selenium, vitamins A, D, E, K, B2, B6, B12, C, lutein, lycopene, α- and β-carotene) were significantly lowered from minor to moderate to major inflammation. The results of the present systematic review indicate that most plasma micronutrients fall as part of the systemic inflammatory response irrespective of acute or chronic injury. Therefore, in the presence of a systemic inflammation, plasma micronutrient concentrations should be interpreted with caution. There are a number of methods applied to adjust plasma micronutrient concentrations to avoid misdiagnosis of deficiency. Alternatively, intracellular measurements appear to obviate the need for such plasma adjustment to assess micronutrient status.

Keywords: CRP C-reactive protein; C-reactive protein; Micronutrient status; Systemic inflammation; Trace elements; vitamins.
"""
# TITABS="Diabetic glucose"
topic_terms=pd.Series([TITABS])

# Extract MeSH Query
tab = create_corr_table(topic_terms, grpm_meshes, model, mesh_embeddings)

threshold = 0.99# set similarity threshold
mesh_query = tab[tab.similarity >= threshold].list2.to_list()
print('\n\nMeSH Query:', mesh_query)
tab

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  4.95it/s]



MeSH Query: []


,list1,list2,similarity
0,\nRelationship between nutritional status and ...,"Carbohydrate Metabolism, Inborn Errors",0.856713


    \item [1)] Although the paper provides a link to a directory containing\textit{ sample queries}, it would be very helpful to provide a few examples of  particularly interesting queries that can be written with this system.

    \item [R:] \textcolor{violet}{Si riferisce ala directori tests e al notebook tests.ipynb. alcuni esempi di query particolarmente interessanti che possono essere scritte con questo sistema.  
    Una query di interesse sul datase nutrientico complessivo potrebbe essere la ricerca di varianti putatiamente correlate a due diverse condizioni nutrizionali assemblando una query composita:
    .
    esempio: Relationship between nutritional status and the systemic inflammatory response: micronutrients a livello genetico \cite{https://pubmed.ncbi.nlm.nih.gov/30220267/}
    .
    Query composita di questo tipo può essere composta con l'ausilio di BioBERT embedding per slezionare MESH partendo da una descrizione testuale (come proposto nel notebook "test.ipynb"), una funzinoe utile per l'implementazione di un interfaccia utente. 
    . 
    }
    % c'è tempo per un ultimaimplemetaione?